# SVD test

### read Movielens data

In [125]:
from collections import defaultdict
from pprint import pprint
from pandas import DataFrame
import pandas as pd

def loadMovieLens(path='data/movielens'):
  # Get movie titles
  movies={}
  # ingnore
  for line in open(path+'/u.item',errors='ignore'):
    (id,title)=line.split('|')[0:2]
    movies[id]=title
  
  # Load data
  prefs=defaultdict(dict)
  for line in open(path+'/u.data'):
    (user,movieid,rating,ts)=line.split('\t')
    prefs[int(user)][int(movieid)]=float(rating)
  return prefs,movies

### drop rows that has to much NaN

In [187]:
# data = DataFrame(movies,index=[0])
# data
prefs,movies = loadMovieLens()
data = DataFrame(prefs)
counts=[n for n in range(1,1682) if data.ix[n].count()>100]
print(len(counts))
cleanedData = data.ix[counts]
cleanedData=cleanedData.fillna(0)

334


### svd test

In [127]:
import numpy as np

U,Sigma,VT=np.linalg.svd(cleanedData)
Squ=[value**2 for value in Sigma]
valueSum = sum(Squ)
print(valueSum)
for n in range(0,len(Sigma)+1,20):
    print("%d: %f" % (n,sum(Squ[:n])/valueSum))

955621.0
0: 0.000000
20: 0.621745
40: 0.692749
60: 0.747882
80: 0.792778
100: 0.830471
120: 0.862321
140: 0.889204
160: 0.912028
180: 0.931400
200: 0.947672
220: 0.961278
240: 0.972500
260: 0.981599
280: 0.988843
300: 0.994392
320: 0.998309


# LFM test 

In [198]:
def LFM(user_items, F, N, alpha=0.01, lam=0.01):
    P,Q = InitModel(user_items, F)
    user_num, item_num = user_items.shape
    for step in range(N):
        for u in range(user_num):
            for i in range(item_num):
                err = user_items[u][i]-np.dot(P[u],Q[i])
                for f in range(F):
                    gp = err*Q[i][f]+lam*P[u][f]
                    gq = err*P[u][f]+lam*Q[i][f]
                    P[u][f] += alpha*gp
                    Q[i][f] += alpha*gq
        err = user_items-np.dot(P,Q.T)
        print("step %d: %f"%(step,np.sum(err**2)))           
    return P,Q
                    
def InitModel(user_items, F):
    user_num, item_num = user_items.shape
    P = np.random.rand(user_num,F)
    Q = np.random.rand(item_num,F)
    return P,Q

def Recommend(user, P, Q):
    rank = dict()
    for f, puf in P[user].items():
        for i, qfi in Q[f].items():
            if i not in rank:
                rank[i] += puf*qfi
    return rank

In [199]:
# data = [[5,5,0,5],[5,0,3,4],[3,4,0,3],[0,0,5,3],[5,4,4,5],[5,4,5,5]]
# arr = np.array(data)
# P,Q = LFM(arr, 3, 1000,alpha=0.01)
arr = np.array(cleanedData)
P,Q = LFM(arr,100,1000)
print(P)
print(Q)
print(np.dot(P,Q.T))

step 0: 2439085.875030
step 1: 1189168.125998
step 2: 593555.821211


KeyboardInterrupt: 

In [189]:
np.array(data)

array([[  5.,   4.,  nan, ...,   5.,  nan,  nan],
       [  3.,  nan,  nan, ...,  nan,  nan,   5.],
       [  4.,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [193]:
print('test')

test
